In [2]:
# Installa pacchetti se non sono già presenti
!pip install -q pandas numpy scikit-learn matplotlib fairlearn qiskit qiskit-machine-learning torch


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, matthews_corrcoef
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.base import BaseEstimator, ClassifierMixin
from qiskit.utils import algorithm_globals
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import torch
from torch import nn
from sklearn.neural_network import MLPClassifier
import warnings

In [4]:
# Ignora warning di precisione indefinita
warnings.filterwarnings("ignore", category=UserWarning)

# 1. Carica il dataset COMPAS filtrando per Caucasian e African-American
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
df = pd.read_csv(url)
df = df[df['race'].isin(['Caucasian', 'African-American'])].copy()

In [5]:
# 2. Crea la variabile sensibile 'race_binary': 1 = African-American, 0 = Caucasian
df['race_binary'] = (df['race'] == 'African-American').astype(int)

In [6]:
# 3. Target: recidiva entro 2 anni
y = df['two_year_recid'].astype(int).values
sensitive_attr = df['race_binary'].values

# 4. Selezione delle feature
features = ['age', 'priors_count', 'juv_fel_count', 'juv_misd_count', 'c_charge_degree']
df = df.copy()
df['c_charge_degree'] = df['c_charge_degree'].apply(lambda x: 1 if x == 'F' else 0)
X = df[features].fillna(0)

# 5. Suddivisione in train/test
X_train, X_test, y_train, y_test, sens_train, sens_test = train_test_split(
    X, y, sensitive_attr, test_size=0.3, random_state=0, stratify=y)

# 6. Normalizzazione
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 7. QNN: costruzione circu# ... (tutto il codice precedente per COMPAS rimane invariato)

# 6. Adversarial Debiasing (Quantum Predictor + Classico Adversary)

class Adversary(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.fc(x)

class QAdvModel(nn.Module):
    def __init__(self, q_model, adv_model, alpha=1.0):
        super().__init__()
        self.q_model = q_model
        self.adv_model = adv_model
        self.alpha = alpha

    def forward(self, x):
        q_out = self.q_model(x)
        adv_out = self.adv_model(q_out.detach())
        return q_out, adv_out

q_model = QuantumClassifier()
adversary = Adversary(1)  # 1 output dalla QNN
q_adv = QAdvModel(q_model, adversary, alpha=1.0)

opt_pred = torch.optim.AdamW(q_adv.q_model.parameters(), lr=0.001)
opt_adv = torch.optim.AdamW(q_adv.adv_model.parameters(), lr=0.001)

sens_train_torch = torch.tensor(sens_train * 1.0, dtype=torch.float32).view(-1, 1)

for epoch in range(100):
    total_loss = 0
    for x_batch, y_batch, s_batch in zip(X_train_torch, y_train_torch, sens_train_torch):
        x_batch = x_batch.view(1, -1)
        y_batch = y_batch.view(1, -1)
        s_batch = s_batch.view(1, -1)

        opt_pred.zero_grad()
        opt_adv.zero_grad()

        pred_out, adv_out = q_adv(x_batch)

        pred_loss = loss_fn(pred_out, y_batch)
        adv_loss = loss_fn(adv_out, s_batch)
        loss = pred_loss - q_adv.alpha * adv_loss

        loss.backward()
        opt_pred.step()
        opt_adv.step()

        total_loss += loss.item()
    print(f"[AdvEpoch {epoch+1}] Loss: {total_loss:.4f}")

# Valutazione del modello adversarial
qadv_scores = q_model(X_test_torch).detach().numpy().flatten()
qadv_pred = (qadv_scores >= 0.5).astype(int)

metrics_qadv = full_metrics(y_test, qadv_pred, qadv_scores, sens_test)
metrics_df.loc['QNN Adversarial'] = metrics_qadv

print("\n=== Confronto con QNN Adversarial Debiasing ===")
print(metrics_df)
ito parametrico
num_qubits = X.shape[1]
x_params = ParameterVector("x", num_qubits)
y_params = ParameterVector("y", num_qubits)
qc = QuantumCircuit(num_qubits)
for i in range(num_qubits):
    qc.ry(x_params[i], i)

# Aggiunta di entanglement e profondità
for _ in range(2):
    for i in range(num_qubits - 1):
        qc.cz(i, i+1)
    for i in range(num_qubits):
        qc.rx(y_params[i], i)
        qc.ry(y_params[i], i)

qnn = EstimatorQNN(
    circuit=qc,
    input_params=x_params,
    weight_params=y_params
)

model = TorchConnector(qnn)

# 8. PyTorch training con mini-batch e più epoche
class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.qnn = model
    def forward(self, x):
        return torch.sigmoid(self.qnn(x))

qc_model = QuantumClassifier()
loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(qc_model.parameters(), lr=0.001)

X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_torch = torch.tensor(y_train * 1.0, dtype=torch.float32).view(-1, 1)
train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

#aumentare le epoche a 200
for epoch in range(10):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = qc_model(x_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# 9. Predizioni su test set (QNN)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32)
y_scores_qnn = qc_model(X_test_torch).detach().numpy().flatten()
y_pred_qnn = (y_scores_qnn >= 0.5).astype(int)

# 10. Dummy model per Fairlearn (QNN)
class QuantumProbModel(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - np.array(y_scores_qnn), np.array(y_scores_qnn)]).T
    def _check_is_fitted(self):
        pass

X_dummy = np.zeros((len(y_test), 1))
postproc_qnn = ThresholdOptimizer(
    estimator=QuantumProbModel(),
    constraints="equalized_odds",
    prefit=True
)
postproc_qnn.fit(X=X_dummy, y=y_test, sensitive_features=sens_test)
y_pred_qnn_fair = postproc_qnn.predict(X=X_dummy, sensitive_features=sens_test)

# 11. Classico MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=500, random_state=0)
mlp.fit(X_train_scaled, y_train)
y_scores_mlp = mlp.predict_proba(X_test_scaled)[:, 1]
y_pred_mlp = mlp.predict(X_test_scaled)

# 12. Equalized Odds post-processing (MLP)
class MLPProbModel(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - y_scores_mlp, y_scores_mlp]).T
    def _check_is_fitted(self):
        pass

postproc_mlp = ThresholdOptimizer(
    estimator=MLPProbModel(),
    constraints="equalized_odds",
    prefit=True
)
postproc_mlp.fit(X=X_dummy, y=y_test, sensitive_features=sens_test)
y_pred_mlp_fair = postproc_mlp.predict(X=X_dummy, sensitive_features=sens_test)

# 13. Metriche
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, matthews_corrcoef

def full_metrics(y_true, y_pred, y_score, sensitive_attr):
    s = np.array(sensitive_attr)
    yt, yp = np.array(y_true), np.array(y_pred)
    priv, unpriv = s == 0, s == 1

    def tpr(y, yhat): return np.mean((yhat == 1) & (y == 1)) / max(np.mean(y == 1), 1e-6)
    def fpr(y, yhat): return np.mean((yhat == 1) & (y == 0)) / max(np.mean(y == 0), 1e-6)
    sr_priv, sr_unpriv = np.mean(yp[priv]), np.mean(yp[unpriv])

    return {
        'Accuracy': accuracy_score(yt, yp),
        'AUC': roc_auc_score(yt, y_score),
        'F1': f1_score(yt, yp, zero_division=0),
        'Precision': precision_score(yt, yp, zero_division=0),
        'Recall': recall_score(yt, yp, zero_division=0),
        'MCC': matthews_corrcoef(yt, yp),
        'EOD': tpr(yt[unpriv], yp[unpriv]) - tpr(yt[priv], yp[priv]),
        'AOD': 0.5 * ((fpr(yt[unpriv], yp[unpriv]) - fpr(yt[priv], yp[priv])) +
                      (tpr(yt[unpriv], yp[unpriv]) - tpr(yt[priv], yp[priv]))),
        'DI': sr_unpriv / sr_priv if sr_priv > 0 else np.nan,
        'SPR': sr_unpriv - sr_priv
    }

metrics_qnn_before = full_metrics(y_test, y_pred_qnn, y_scores_qnn, sens_test)
metrics_qnn_after = full_metrics(y_test, y_pred_qnn_fair, y_scores_qnn, sens_test)
metrics_mlp_before = full_metrics(y_test, y_pred_mlp, y_scores_mlp, sens_test)
metrics_mlp_after = full_metrics(y_test, y_pred_mlp_fair, y_scores_mlp, sens_test)

# 14. Visualizzazione
metrics_df = pd.DataFrame([
    metrics_qnn_before,
    metrics_qnn_after,
    metrics_mlp_before,
    metrics_mlp_after
], index=["QNN Before", "QNN After", "MLP Before", "MLP After"])
print(metrics_df)


Epoch 1, Loss: 0.6932
Epoch 2, Loss: 0.6623
Epoch 3, Loss: 0.6750
Epoch 4, Loss: 0.6607
Epoch 5, Loss: 0.7244
Epoch 6, Loss: 0.7004
Epoch 7, Loss: 0.6667
Epoch 8, Loss: 0.6981
Epoch 9, Loss: 0.6522
Epoch 10, Loss: 0.6421


/opt/miniconda3/envs/qiskit-env/lib/python3.10/site-packages/fairlearn/postprocessing/_interpolated_thresholder.py:154: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.81908095e-04 9.99307350e-01 1.81908095e-04 9.99307350e-01
 9.99307350e-01 1.81908095e-04 1.81908095e-04 1.81908095e-04
 9.99307350e-01 9.99307350e-01 1.81908095e-04 9.99307350e-01
 1.81908095e-04 1.81908095e-04 1.81908095e-04 1.81908095e-04
 1.81908095e-04 1.81908095e-04 7.49400000e-02 1.81908095e-04
 1.81908095e-04 1.81908095e-04 1.81908095e-04 1.81908095e-04
 1.81908095e-04 9.99307350e-01 1.81908095e-04 9.99307350e-01
 1.81908095e-04 1.81908095e-04 1.81908095e-04 9.99307350e-01
 1.81908095e-04 1.81908095e-04 9.99307350e-01 7.49400000e-02
 9.99307350e-01 1.81908095e-04 1.81908095e-04 1.81908095e-04
 7.49400000e-02 7.49400000e-02 7.49400000e-02 9.99307350e-01
 7.49400000e-02 7.49400000e-02 9.99307350e-01 7.49400000e-02
 9.99307350e-01 9.9

            Accuracy       AUC        F1  Precision    Recall       MCC  \
QNN Before  0.618970  0.675406  0.422350   0.719888  0.298837  0.249167   
QNN After   0.650407  0.675406  0.563895   0.673667  0.484884  0.295622   
MLP Before  0.679675  0.731260  0.627599   0.685007  0.579070  0.353823   
MLP After   0.672087  0.731260  0.615873   0.678322  0.563953  0.338362   

                 EOD       AOD        DI       SPR  
QNN Before  0.141548  0.106637  1.987752  0.119563  
QNN After   0.007500  0.002856  1.084715  0.027034  
MLP Before  0.239167  0.200759  1.877780  0.225784  
MLP After  -0.004167 -0.002692  1.071739  0.026643  


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, matthews_corrcoef
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.base import BaseEstimator, ClassifierMixin
from qiskit.utils import algorithm_globals
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import torch
from torch import nn
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# 1. Caricamento e pre-processing COMPAS
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
df = pd.read_csv(url)
df = df[df['race'].isin(['Caucasian', 'African-American'])].copy()
df['race_binary'] = (df['race'] == 'African-American').astype(int)
df['c_charge_degree'] = df['c_charge_degree'].apply(lambda x: 1 if x == 'F' else 0)

features = ['age', 'priors_count', 'juv_fel_count', 'juv_misd_count', 'c_charge_degree']
X = df[features].fillna(0)
y = df['two_year_recid'].astype(int).values
sensitive_attr = df['race_binary'].values

X_train, X_test, y_train, y_test, sens_train, sens_test = train_test_split(
    X, y, sensitive_attr, test_size=0.3, random_state=0, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. QNN come regressione logistica quantistica
num_qubits = X.shape[1]
x_params = ParameterVector("x", num_qubits)
y_params = ParameterVector("y", num_qubits)
qc = QuantumCircuit(num_qubits)
for i in range(num_qubits):
    qc.ry(x_params[i], i)
for _ in range(2):
    for i in range(num_qubits - 1):
        qc.cz(i, i+1)
    for i in range(num_qubits):
        qc.rx(y_params[i], i)
        qc.ry(y_params[i], i)

qnn = EstimatorQNN(
    circuit=qc,
    input_params=x_params,
    weight_params=y_params
)

model = TorchConnector(qnn)

class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.qnn = model
    def forward(self, x):
        return torch.sigmoid(self.qnn(x))

qc_model = QuantumClassifier()
loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(qc_model.parameters(), lr=0.001)

X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_torch = torch.tensor(y_train * 1.0, dtype=torch.float32).view(-1, 1)
train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

for epoch in range(10):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = qc_model(x_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32)
y_scores_qnn = qc_model(X_test_torch).detach().numpy().flatten()
y_pred_qnn = (y_scores_qnn >= 0.5).astype(int)

class QuantumProbModel(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - np.array(y_scores_qnn), np.array(y_scores_qnn)]).T
    def _check_is_fitted(self):
        pass

X_dummy = np.zeros((len(y_test), 1))
postproc_qnn = ThresholdOptimizer(
    estimator=QuantumProbModel(),
    constraints="equalized_odds",
    prefit=True
)
postproc_qnn.fit(X=X_dummy, y=y_test, sensitive_features=sens_test)
y_pred_qnn_fair = postproc_qnn.predict(X=X_dummy, sensitive_features=sens_test)

# 3. MLP
mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=500, random_state=0)
mlp.fit(X_train_scaled, y_train)
y_scores_mlp = mlp.predict_proba(X_test_scaled)[:, 1]
y_pred_mlp = mlp.predict(X_test_scaled)

class MLPProbModel(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - y_scores_mlp, y_scores_mlp]).T
    def _check_is_fitted(self):
        pass

postproc_mlp = ThresholdOptimizer(
    estimator=MLPProbModel(),
    constraints="equalized_odds",
    prefit=True
)
postproc_mlp.fit(X=X_dummy, y=y_test, sensitive_features=sens_test)
y_pred_mlp_fair = postproc_mlp.predict(X=X_dummy, sensitive_features=sens_test)

# Nota: la QNN nel nostro modello agisce come una regressione logistica quantistica
# perché produce una probabilità tramite funzione sigmoide e viene ottimizzata
# con Binary Cross Entropy Loss (BCELoss), esattamente come una logistic regression classica.

# 4. Logistic Regression classica
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train_scaled, y_train)
y_scores_logreg = logreg.predict_proba(X_test_scaled)[:, 1]
y_pred_logreg = logreg.predict(X_test_scaled)

class LogRegProbModel(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - y_scores_logreg, y_scores_logreg]).T
    def _check_is_fitted(self):
        pass

postproc_logreg = ThresholdOptimizer(
    estimator=LogRegProbModel(),
    constraints="equalized_odds",
    prefit=True
)
postproc_logreg.fit(X=X_dummy, y=y_test, sensitive_features=sens_test)
y_pred_logreg_fair = postproc_logreg.predict(X=X_dummy, sensitive_features=sens_test)

# 5. Metriche

def full_metrics(y_true, y_pred, y_score, sensitive_attr):
    s = np.array(sensitive_attr)
    yt, yp = np.array(y_true), np.array(y_pred)
    priv, unpriv = s == 0, s == 1

    def tpr(y, yhat): return np.mean((yhat == 1) & (y == 1)) / max(np.mean(y == 1), 1e-6)
    def fpr(y, yhat): return np.mean((yhat == 1) & (y == 0)) / max(np.mean(y == 0), 1e-6)
    sr_priv, sr_unpriv = np.mean(yp[priv]), np.mean(yp[unpriv])

    return {
        'Accuracy': accuracy_score(yt, yp),
        'AUC': roc_auc_score(yt, y_score),
        'F1': f1_score(yt, yp, zero_division=0),
        'Precision': precision_score(yt, yp, zero_division=0),
        'Recall': recall_score(yt, yp, zero_division=0),
        'MCC': matthews_corrcoef(yt, yp),
        'EOD': tpr(yt[unpriv], yp[unpriv]) - tpr(yt[priv], yp[priv]),
        'AOD': 0.5 * ((fpr(yt[unpriv], yp[unpriv]) - fpr(yt[priv], yp[priv])) +
                      (tpr(yt[unpriv], yp[unpriv]) - tpr(yt[priv], yp[priv]))),
        'DI': sr_unpriv / sr_priv if sr_priv > 0 else np.nan,
        'SPR': sr_unpriv - sr_priv
    }

metrics_qnn_before = full_metrics(y_test, y_pred_qnn, y_scores_qnn, sens_test)
metrics_qnn_after = full_metrics(y_test, y_pred_qnn_fair, y_scores_qnn, sens_test)
metrics_mlp_before = full_metrics(y_test, y_pred_mlp, y_scores_mlp, sens_test)
metrics_mlp_after = full_metrics(y_test, y_pred_mlp_fair, y_scores_mlp, sens_test)
metrics_logreg_before = full_metrics(y_test, y_pred_logreg, y_scores_logreg, sens_test)
metrics_logreg_after = full_metrics(y_test, y_pred_logreg_fair, y_scores_logreg, sens_test)

metrics_df = pd.DataFrame([
    metrics_qnn_before,
    metrics_qnn_after,
    metrics_mlp_before,
    metrics_mlp_after,
    metrics_logreg_before,
    metrics_logreg_after
], index=["QNN Before", "QNN After", "MLP Before", "MLP After", "LogReg Before", "LogReg After"])

print(metrics_df)


Epoch 1, Loss: 0.7074
Epoch 2, Loss: 0.7092
Epoch 3, Loss: 0.7409
Epoch 4, Loss: 0.7052
Epoch 5, Loss: 0.7149
Epoch 6, Loss: 0.6863
Epoch 7, Loss: 0.6740
Epoch 8, Loss: 0.6471
Epoch 9, Loss: 0.6797
Epoch 10, Loss: 0.6693


/opt/miniconda3/envs/qiskit-env/lib/python3.10/site-packages/fairlearn/postprocessing/_interpolated_thresholder.py:154: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.         1.         0.         0.00147239 0.00147239 0.
 0.         0.         0.00147239 0.00147239 0.         0.00147239
 0.         0.         0.         0.         0.         0.
 0.00147239 0.         0.00147239 0.         0.         0.00147239
 0.         0.         0.00147239 0.00147239 0.00147239 0.
 0.         0.00147239 0.00147239 0.         1.         0.
 1.         0.         0.         0.         1.         0.00147239
 1.         1.         1.         1.         1.         0.00147239
 1.         1.         0.         0.         0.00147239 0.
 0.00147239 0.         0.00147239 0.00147239 1.         0.00147239
 1.         0.00147239 0.         0.00147239 1.         0.
 0.         0.         0.00147239 0.         0.00147239 0.0014

               Accuracy       AUC        F1  Precision    Recall       MCC  \
QNN Before     0.600000  0.651300  0.584927   0.566449  0.604651  0.200132   
QNN After      0.608672  0.651300  0.414911   0.684492  0.297674  0.220722   
MLP Before     0.679675  0.731260  0.627599   0.685007  0.579070  0.353823   
MLP After      0.670461  0.731260  0.616646   0.673554  0.568605  0.334928   
LogReg Before  0.675881  0.726159  0.620558   0.682961  0.568605  0.346155   
LogReg After   0.673713  0.726159  0.607562   0.691395  0.541860  0.342597   

                    EOD       AOD        DI       SPR  
QNN Before     0.196548  0.155187  1.422462  0.167358  
QNN After      0.001548  0.001091  1.086877  0.016730  
MLP Before     0.239167  0.200759  1.877780  0.225784  
MLP After      0.002976  0.007152  1.098231  0.036482  
LogReg Before  0.243571  0.207371  1.936355  0.231835  
LogReg After   0.002857 -0.006491  1.061766  0.021750  


In [2]:
!pip install --upgrade fairlearn

In [ ]:
# ... (tutto il codice precedente per COMPAS rimane invariato)

# 6. Adversarial Debiasing (Quantum Predictor + Classico Adversary)

class Adversary(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.fc(x)

class QAdvModel(nn.Module):
    def __init__(self, q_model, adv_model, alpha=1.0):
        super().__init__()
        self.q_model = q_model
        self.adv_model = adv_model
        self.alpha = alpha

    def forward(self, x):
        q_out = self.q_model(x)
        adv_out = self.adv_model(q_out.detach())
        return q_out, adv_out

q_model = QuantumClassifier()
adversary = Adversary(1)  # 1 output dalla QNN
q_adv = QAdvModel(q_model, adversary, alpha=1.0)

opt_pred = torch.optim.AdamW(q_adv.q_model.parameters(), lr=0.001)
opt_adv = torch.optim.AdamW(q_adv.adv_model.parameters(), lr=0.001)

sens_train_torch = torch.tensor(sens_train * 1.0, dtype=torch.float32).view(-1, 1)

for epoch in range(100):
    total_loss = 0
    for x_batch, y_batch, s_batch in zip(X_train_torch, y_train_torch, sens_train_torch):
        x_batch = x_batch.view(1, -1)
        y_batch = y_batch.view(1, -1)
        s_batch = s_batch.view(1, -1)

        opt_pred.zero_grad()
        opt_adv.zero_grad()

        pred_out, adv_out = q_adv(x_batch)

        pred_loss = loss_fn(pred_out, y_batch)
        adv_loss = loss_fn(adv_out, s_batch)
        loss = pred_loss - q_adv.alpha * adv_loss

        loss.backward()
        opt_pred.step()
        opt_adv.step()

        total_loss += loss.item()
    print(f"[AdvEpoch {epoch+1}] Loss: {total_loss:.4f}")

# Valutazione del modello adversarial
qadv_scores = q_model(X_test_torch).detach().numpy().flatten()
qadv_pred = (qadv_scores >= 0.5).astype(int)

metrics_qadv = full_metrics(y_test, qadv_pred, qadv_scores, sens_test)
metrics_df.loc['QNN Adversarial'] = metrics_qadv

# Equalized Odds Postprocessing su QNN Adversarial
class QAdvProbModel(BaseEstimator, ClassifierMixin):
    def fit(self, X, y=None):
        self._is_fitted = True
        return self
    def predict_proba(self, X):
        return np.vstack([1 - qadv_scores, qadv_scores]).T
    def _check_is_fitted(self):
        pass

from fairlearn.postprocessing import ThresholdOptimizer

post_qadv = ThresholdOptimizer(
    estimator=QAdvProbModel(),
    constraints="equalized_odds",
    prefit=True
)
post_qadv.fit(X=np.zeros_like(qadv_scores).reshape(-1, 1), y=y_test, sensitive_features=sens_test)
qadv_fair = post_qadv.predict(X=np.zeros_like(qadv_scores).reshape(-1, 1), sensitive_features=sens_test)

metrics_qadv_fair = full_metrics(y_test, qadv_fair, qadv_scores, sens_test)
metrics_df.loc['QNN Adv + EO'] = metrics_qadv_fair

print("\n=== Confronto con QNN Adversarial Debiasing + Equalized Odds ===")
print(metrics_df)


[AdvEpoch 1] Loss: -54127.7640
